Clone the tutorial repo

In [8]:
import world
import utils
from world import cprint
import torch
import numpy as np
from tensorboardX import SummaryWriter
import time
import Procedure
from os.path import join
# ==============================
utils.set_seed(world.seed)
print(">>SEED:", world.seed)
# ==============================
import register
from register import dataset
from torch import nn

>>SEED: 2020
loading [../data/gowalla]
810128 interactions for training
217242 interactions for testing
gowalla Sparsity : 0.0008396216228570436
gowalla is ready to go
===========config================
{'A_n_fold': 100,
 'A_split': False,
 'K': 4,
 'bigdata': False,
 'bpr_batch_size': 2048,
 'decay': 0.0001,
 'dropout': 0,
 'dual_res': False,
 'keep_prob': 0.6,
 'latent_dim_rec': 64,
 'learnable_time': False,
 'lightGCN_n_layers': 3,
 'lr': 0.001,
 'lr_time': 0.0001,
 'multicore': 0,
 'pretrain': 0,
 'pretrained_file_name': 'ltocf',
 'solver': 'euler',
 'test_u_batch_size': 100,
 'time_split': 4}
cores for test: 32
comment: lt-ncf
tensorboard: 1
LOAD: 0
Weight path: ./checkpoints
Test Topks: [20]
using bpr loss
adjoint method: False
===========end===================


In [9]:
Recmodel = register.MODELS[world.model_name](world.config, dataset)
if world.parallel == True:
    Recmodel = nn.DataParallel(Recmodel, device_ids=[0, 1])
Recmodel = Recmodel.to(world.device)
if world.config['learnable_time'] == False:
    bpr = utils.BPRLoss(Recmodel, world.config)
elif world.config['learnable_time'] == True:
    bpr_t = utils.BPRLossT(Recmodel, world.config)

weight_file = utils.getFileName()
pretrained_weight_file = utils.getPretrainedFileName(world.config['pretrained_file_name'])
print(f"load and save to {pretrained_weight_file}")
if world.LOAD:
    try:
        Recmodel.load_state_dict(torch.load(pretrained_weight_file,map_location=torch.device('cpu')))
        world.cprint(f"loaded model weights from {pretrained_weight_file}")
    except FileNotFoundError:
        print(f"{pretrained_weight_file} not exists, start from beginning")
Neg_k = 1

save_name = time.strftime("%m-%d-%Hh%Mm-") + "-" + world.dataset + "-" + world.model_name + "-" + world.config['solver'] + "-adjoint_" + str(world.adjoint) + "-learnable_t_" + str(world.config['learnable_time']) + "-dual_res_" + str(world.config['dual_res']) + "-lr" + str(world.config['lr']) + "-lr_time" + str(world.config['lr_time']) + "-decay" + str(world.config['decay'])+ "-" + world.comment

# init tensorboard
if world.tensorboard:
    w : SummaryWriter = SummaryWriter(join(world.BOARD_PATH, save_name))
else:
    w = None
    world.cprint("not enable tensorflowboard")


use NORMAL distribution initilizer
loading adjacency matrix
successfully loaded...
don't split the matrix
lgn is already to go(dropout:0)
load and save to /home/bigdyl/jeongwhanchoi/tutorial21/LT-OCF/code/pretrain/ltocf


In [12]:
for epoch in range(world.TRAIN_epochs+1):
    start = time.time()
    if epoch %10 == 0:
        cprint("[TEST]")
        Procedure.Test(dataset, Recmodel, epoch, w, world.config['multicore'])

    if world.model_name == 'ltocf':
        if world.config['learnable_time'] == False:
            output_information = Procedure.BPR_train_original(dataset, Recmodel, bpr, epoch, neg_k=Neg_k,w=w)
            print(f'EPOCH[{epoch+1}/{world.TRAIN_epochs}] {output_information}')
        else:
            output_information, times_list= Procedure.BPR_train_ode(dataset, Recmodel, bpr_t, epoch, neg_k=Neg_k,w=w)
            print(f'EPOCH[{epoch+1}/{world.TRAIN_epochs}] {output_information}')
            print(times_list)

[TEST]
Inference time: 6.3403s
{'precision': array([0.00020932]), 'recall': array([0.00060648]), 'ndcg': array([0.00044858])}
Train time: 11.0391s
EPOCH[1/1000] loss0.492-|Sample:0.08|
Train time: 11.0365s
EPOCH[2/1000] loss0.229-|Sample:0.08|
Train time: 11.0462s
EPOCH[3/1000] loss0.168-|Sample:0.09|
Train time: 11.0316s
EPOCH[4/1000] loss0.137-|Sample:0.09|
Train time: 11.0465s
EPOCH[5/1000] loss0.120-|Sample:0.09|
Train time: 11.0574s
EPOCH[6/1000] loss0.107-|Sample:0.09|
Train time: 11.0497s
EPOCH[7/1000] loss0.098-|Sample:0.09|
Train time: 11.0687s
EPOCH[8/1000] loss0.091-|Sample:0.09|
Train time: 11.0566s
EPOCH[9/1000] loss0.086-|Sample:0.09|
Train time: 11.0879s
EPOCH[10/1000] loss0.082-|Sample:0.09|
[TEST]
Inference time: 5.7959s
{'precision': array([0.03506598]), 'recall': array([0.11539871]), 'ndcg': array([0.09609495])}
Train time: 11.1050s
EPOCH[11/1000] loss0.078-|Sample:0.08|
Train time: 11.1075s
EPOCH[12/1000] loss0.075-|Sample:0.09|
Train time: 11.1397s
EPOCH[13/1000] l

KeyboardInterrupt: 